**LEITURA E FORMATAÇÃO DO DATASET**

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
filepath = 'emprestimos.txt'
filenames = []
with open(filepath) as fp:  
   line = fp.readline()
   cnt = 1
   while line:
      filenames.append(line)
      line = fp.readline()
      cnt += 1

semestres = (nome[-6:].rstrip('\n').strip('-') for nome in filenames)
l_empr = [pd.read_csv(name, sep=';') for name in filenames] # lista de dataframes de cada semestre
for df in l_empr:
  df['semestre'] = next(semestres)  # lista de dataframes de cada semestre
  print(df['codigo_barras'].value_counts().head(1))
emprestimo_bruto = pd.concat(l_empr,axis=0) # Concat neles
exemplares_arcevo =  pd.read_csv('http://bit.do/exemplares_arcervo', sep=';')
exemplares_sem_barra = pd.read_csv('http://bit.do/arcevo', sep=';')
exemplares_com_barra = pd.read_csv('http://bit.do/exemplares', sep=';')
exemplares_merge = pd.merge (exemplares_arcevo,exemplares_com_barra, on = 'registro_sistema',how='inner')
emprestimo_bruto['data_emprestimo']= pd.to_datetime(emprestimo_bruto['data_emprestimo'], errors='coerce')
emprestimo_bruto['data_devolucao']= pd.to_datetime(emprestimo_bruto['data_devolucao'], errors='coerce')
emprestimo_bruto['data_renovacao']= pd.to_datetime(emprestimo_bruto['data_devolucao'], errors='coerce')
emprestimo_bruto.dropna(axis=0,subset=['data_devolucao'],inplace=True)
emprestimo_bruto['tempo_pra_devolver'] = emprestimo_bruto['data_devolucao'] -emprestimo_bruto['data_emprestimo']
emprestimo = pd.merge(emprestimo_bruto,exemplares_merge, on = 'codigo_barras',how= 'left') # mergem do emprestimo com os exemplares pra vincular os nomes dos livros aos empréstimos
emprestimo.set_index('data_emprestimo',inplace=True)
emprestimo.sort_index(inplace = True,ascending = False)



**INFO SOBRE OS DATA SETS**


In [0]:
#emprestimo_bruto['renovado'] =  emprestimo_bruto['data_renovacao'].isnull()

#emprestimo_bruto['atrasado'] = (emprestimo_bruto['tempo_pra_devolver'] > '15 days' & emprestimo_bruto['renovado'] == False) | (emprestimo_bruto['tempo_pra_devolver'] > '30 days' & emprestimo_bruto['renovado'] ==True)
print('1')
print(emprestimo_bruto.info())
print('2')
print(exemplares_arcevo.info())
print('3')
print(exemplares_sem_barra.info())
print('4')
print(exemplares_com_barra.info())



**TIME SAMPLE POR MÊS EM CADA SEMESTRE E CONTAGEM DE EMPRÉSTIMO**

In [0]:
from google.colab import files

emprestimolist_sem = list()
semestres = (nome[-6:].rstrip('\n').strip('-') for nome in filenames)

for s in semestres:
  emprestimolist_sem.append(emprestimo[emprestimo['semestre'] == s])
emprestimolist_sem_mes = list()
for df in emprestimolist_sem:
  emprestimolist_sem_mes.append(df.resample('M').count())

emprestimolist_sem_mes_rear = list()
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (14, 8))
for df in emprestimolist_sem_mes:
  df.index =  np.arange(1,len(df)+1)
  emprestimolist_sem_mes_rear.append(df)
for df in emprestimolist_sem_mes_rear:
  df.plot(y='id_emprestimo',use_index=True,ax=ax)
  plt.savefig('grafico_semestres.png',format = 'png')
  files.download('grafico_semestres.png')
test = plt.figure()
ax.legend(['20121','20122','20131','20132','20141','20142','20151','20152','20161','20162','20171','20172'])
ax.set_title('EMPRESTIMO AO DECORRER DO SEMESTRE')
ax.set_xlabel('Representacao ordinal do mês em relação ao semestre')
ax.set_ylabel('quantidade de emprestimos')
#plt.savefig('grafico_semestresaa.png',format = 'png')
#files.download('grafico_semestresaa.png')
plt.show()

In [0]:
semestres = (nome[-6:].rstrip('\n').strip('-') for nome in filenames)
for df in emprestimolist_sem_mes_rear:
  test = plt.figure()
  s = next(semestres)
  df.plot(y='id_emprestimo',use_index=True,figsize = (14, 8),legend=False,)
  plt.title('EMPRESTIMO AO DECORRER DO SEMESTRE '+s)
  plt.xlabel('Representacao ordinal do mês em relação ao semestre')
  plt.ylabel('quantidade de emprestimos')
  plt.savefig('grafico_semestres'+s+'.png',format = 'png')
  files.download('grafico_semestres'+s+'.png')
  plt.show()